In [ ]:
import asyncio
import base64
import hashlib
import hmac
import json
import time
import websockets
import sqlite3
import pandas as pd
import numpy as np
import json


btcturk_responses = []  # List to store BTCTurk responses
binance_responses = []  # List to store Binance responses

async def connect_binance():
    async with websockets.connect('wss://stream.binance.com:9443/ws/btcusdt@depth') as websocket:
        while True:
            response = await websocket.recv()  
            
            response = json.loads(response)
            binance_responses.clear()
            binance_responses.append([response['a'], response['b']])  # Store Binance response in the list

class InterfaceWS:
    def __init__(self, exchange_name: str = None) -> None:
        self.exchange_name = "BTC Turk"
        self.uri = "wss://ws-feed-pro.btcturk.com/"

    async def authenticate_ws(self) -> bool:
        self.ws = await websockets.connect(self.uri)
        publicKey = '52e20623-5d38-4192-b33d-bc8d3276c9c6'
        privateKey = 'PlfIjwMjsDnrYwq/3YnNCS3n13Rq43ye'
        nonce = 3000
        baseString = "{}{}".format(publicKey, nonce).encode("utf-8")
        signature = hmac.new(
            base64.b64decode(privateKey), baseString, hashlib.sha256
        ).digest()
        signature = base64.b64encode(signature)
        timestamp = round(time.time() * 1000)
        hmacMessageObject = [
            151,
            {
                "type": 151,
                "channel": 'orderbook',
                "event": 'BTCUSDT',
                "join": True
            }
        ]
        await self.ws.send(json.dumps(hmacMessageObject))
        while True:
            try:
                response = await asyncio.wait_for(self.ws.recv(), timeout=0.5)
                response = json.loads(response)
                
                btcturk_responses.clear()
                btcturk_responses.append([response[1]['AO'],response[1]['BO']]) 
                
            except Exception as e:
                print(e)
                
 

async def main():
    tasks = [connect_binance(), InterfaceWS().authenticate_ws()]
    await asyncio.gather(*tasks)
    

def strategy(type_ ,ask, bid):
    break_out_flag = False

    for index, row in ask.iterrows():

        row_amount = row['amount']

        for j in range(len(bid)):

            if bid.iloc[j]['price'] > row['price']:

                if row_amount == 0:

                    break

                elif bid.iloc[j]['amount'] == 0:

                    continue

                else:

                    amount= min(bid.iloc[j]['amount'] , row['amount'])

                    row_amount -= amount

                    bid.iloc[j]['amount']  -= amount

                    profit = bid.iloc[j]['price'] - row['price']
                    
                    conn = sqlite3.connect('arbitraj_data.db')
                    c = conn.cursor()

                    c.execute("INSERT INTO arbitraj_test (TIME,TYPE,BUY_PRICE ,SELL_PRICE,PROFIT_UNIT, VOLUME,  PROFIT) VALUES (?, ?, ?, ?, ?, ?, ?)",
                              (time.strftime('%Y:%m:%d %H:%M:%S'),type_ ,row['price'], bid.iloc[j]['price'], profit  , amount, profit*amount ) )
                    conn.commit()
                    
                    conn.close()


            else:

                break_out_flag = True 

                break

        bid = bid[~(bid['amount'] == 0)].copy(deep=True)

        if break_out_flag:
            break 



async def arbitrage_trading():
    while True:
        if btcturk_responses and binance_responses:
            
            binance_ask = pd.DataFrame(binance_responses[0][0]).rename(columns={0 :'price' , 1: 'amount' } )

            binance_ask = binance_ask.astype(float)

            binance_bid = pd.DataFrame(binance_responses[0][1]).rename(columns={0 :'price' , 1: 'amount' }  )

            binance_bid = binance_bid.astype(float)           
            
            
            btcturk_bid= pd.DataFrame.from_dict(btcturk_responses[0][1]).rename(columns={'A' :'amount' , 'P': 'price' }  )

            btcturk_bid = btcturk_bid.astype(float)

            btcturk_ask = pd.DataFrame.from_dict(btcturk_responses[0][0]).rename(columns={'A' :'amount' , 'P': 'price' }  )

            btcturk_ask = btcturk_ask.astype(float)                  
        
            
            strategy('btcturk buy-binance sell',btcturk_ask,binance_bid)
            strategy('binance buy-btcturk sell',binance_ask,btcturk_bid)       
                
                

        await asyncio.sleep(0)  # No delay

if __name__ == "__main__":
    loop = asyncio.get_event_loop()

    main_task = loop.create_task(main())
    arbitrage_task = loop.create_task(arbitrage_trading())

    try:
        loop.run_until_complete(asyncio.gather(main_task, arbitrage_task))
    except KeyboardInterrupt:
        pass
    finally:
        main_task.cancel()
        arbitrage_task.cancel()
        loop.close()
        